# COVID-19 INDIA DASHBOARD

In [4]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import geopandas as gpd
import altair as alt
alt.renderers.enable('default')
alt.themes.enable('dark')
# alt.renderers.enable(renderer_name, embed_options={'renderer': 'svg'})
import json

In [ ]:
country_df = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')
latest_df = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')
vaccine_df = pd.read_csv('http://api.covid19india.org/csv/latest/cowin_vaccine_data_statewise.csv')

country_df.drop(['Date'], axis = 1,inplace=True)

country_df.rename(columns={'Date_YMD':'Date'},inplace=True)

country_df["Date"]= pd.to_datetime(country_df["Date"])
# country_df.set_index('Date')

## Confirmed cases spread trend

In [ ]:
hover = alt.selection_single(on='mouseover',nearest=True, fields=['Date'])
line = alt.Chart().mark_line().encode(x='Date:T',y='Daily Confirmed',color=alt.value('teal')).properties(width=780, height=400).interactive()

point = line.mark_point().encode(opacity=alt.value(0.0)).properties(selection=hover)

text = alt.Chart().mark_text(align='right', dy=-2, fontSize=20).encode(x='Date:T',y='Daily Confirmed',text='Daily Confirmed',detail='Daily Confirmed',color=alt.value('white'),opacity=alt.condition(hover,alt.value(0.8), alt.value(0.0)))

alt.layer(line,point,text,data=country_df).configure(background='black').interactive().configure_axis(grid=False).configure_view(strokeWidth=0)

## Recovery and Fatality spread trends

##### Select an option

In [ ]:
def cases(y_value,colour):
    hover = alt.selection_single(on='mouseover',nearest=True, fields=['Date'])
    line = alt.Chart().mark_line().encode(x='Date:T',y=y_value,color=alt.value(colour)).properties(width=690, height=370).interactive()

    point = line.mark_point().encode(opacity=alt.value(0.0)).properties(selection=hover)

    text = alt.Chart().mark_text(align='right', dy=-2, fontSize=20).encode(x='Date:T',y=y_value,text=y_value,detail=y_value,color=alt.value('white'),opacity=alt.condition(hover,alt.value(0.8), alt.value(0.0)))

    chart = alt.layer(line,point,text,data=country_df).configure(background='black').interactive().configure_axis(grid=False).configure_view(strokeWidth=0)
    return chart

t = widgets.ToggleButtons(
    options=['Recovered', 'Deceased'],
    description='Cases',
    disabled=False,
    value=None,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Recovered Cases', 'Deceased Cases'],
#     icons=['check'] * 3
)


out = widgets.Output(width='800',height='100%')

def toggle_change(change):
    if change.new == 'Recovered':
        out.clear_output()
        with out:
            display(cases(y_value='Daily Recovered', colour = 'green'))
        return out
    elif change.new == 'Deceased':
        out.clear_output()
        with out:
            display(cases(y_value='Daily Deceased', colour = 'red'))
        return out

t.observe(toggle_change, names='value')

out2 = widgets.Output(width='100%',height='100%')
with out2:
    display(t)
box = widgets.VBox(children=[out2, out])
display(box)

## Normalized stacked area plot

In [ ]:
stack = country_df[['Date','Daily Confirmed','Daily Recovered', 'Daily Deceased']]

stack.set_index('Date',inplace=True)

stack.columns = ['Confirmed','Recovered','Deceased']

stack = stack.reset_index()
stack = pd.melt(stack, id_vars='Date', var_name='Cases', value_name= 'Count' )

In [ ]:
hover = alt.selection_single(on='mouseover')
scale = alt.Scale(domain=['Confirmed', 'Recovered', 'Deceased'], range=['#17c8cf', '#20c730', 'red'])
alt.Chart(stack).mark_area().encode(
alt.X('Date:T'),
alt.Y('sum(Count):Q', stack='normalize'),
color=alt.Color('Cases:N',scale = scale),opacity=alt.condition(hover, alt.value(0.8), alt.value(0.4))
).properties(
    width=800,
    height=400,
    selection=hover
).configure(background='black').configure_legend(
  orient='bottom'
).interactive()

## Choropleth map of confirmed cases

##### Hover you cursor over the states 

In [ ]:
latest_df = latest_df[['State','Confirmed','Recovered','Deaths','Active']]

latest_df.set_index('State',inplace=True)
latest_df.drop('Total', inplace=True)

state_list = latest_df.index.tolist()
idx = state_list.index('Jammu and Kashmir')
state_list[idx] = 'Jammu & Kashmir'
latest_df.index = state_list

latest_df.reset_index(inplace=True)

latest_df.rename({'index':'State'},axis=1,inplace=True)

latest_df['Recovery_rate'] = latest_df['Recovered']/latest_df['Confirmed']
latest_df['Death_rate'] = latest_df['Deaths']/latest_df['Confirmed']

gdf = gpd.read_file('Admin2.shp')

gdf = gdf.merge(latest_df,left_on='ST_NM', right_on='State', how='left')

choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values = choro_json['features'])

def gen_map(geodata, color_column, title, tooltip, color_scheme ,scheme_title):
    base = alt.Chart(geodata, title = title).mark_geoshape(stroke='black',strokeWidth=1).encode().properties(width=700, height=800)
    choro = alt.Chart(geodata).mark_geoshape(stroke='white').encode(alt.Color(color_column, type='quantitative',scale=alt.Scale(scheme=color_scheme),title=scheme_title), tooltip=tooltip)
    return base+choro

In [ ]:
india_map = gen_map(geodata=choro_data, color_column='properties.Confirmed', title = "Covid-19 cases", tooltip=['properties.ST_NM:N','properties.Confirmed:Q'], color_scheme='reds',scheme_title='Confirmed cases').configure(background='black')
display(india_map)

## Choropleth map of Active cases, Recovery rate & Death rate

##### Select an option and hover over the map

In [ ]:
t2 = widgets.ToggleButtons(
    options=['Active','Recovery Rate', 'Death Rate'],
    description='India\'s Covid-19 ',
    disabled=False,
    value = None,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Choropleth map of active cases','Choropleth map of recovery rate', 'Choropleth map of death rate'],
#     icons=['check'] * 3
)

out3 = widgets.Output(width='800',height='100%')

def toggle_change2(change):
    if change.new == 'Active':
        out3.clear_output()
        with out3:
            display(gen_map(geodata=choro_data, color_column='properties.Active', title = "Covid-19 cases", tooltip=['properties.ST_NM:N','properties.Active:Q'], color_scheme='lighttealblue',scheme_title='Active cases').configure(background='black'))
        return out3
    elif change.new == 'Recovery Rate':
        out3.clear_output()
        with out3:
            display(gen_map(geodata=choro_data, color_column='properties.Recovery_rate', title = "Covid-19 cases", tooltip=['properties.ST_NM:N', 'properties.Recovered:Q', 'properties.Recovery_rate:Q'], color_scheme='yellowgreen',scheme_title='Recovery rate').configure(background='black'))
        return out3
    elif change.new == 'Death Rate':
        out3.clear_output()
        with out3:
            display(gen_map(geodata=choro_data, color_column='properties.Death_rate', title = "Covid-19 cases", tooltip=['properties.ST_NM:N', 'properties.Deaths:Q', 'properties.Death_rate:Q'], color_scheme='purplered',scheme_title='Death rate').configure(background='black'))
        return out3

t2.observe(toggle_change2, names='value')

out4 = widgets.Output(width='100%',height='100%')
with out4:
    display(t2)
box2 = widgets.VBox(children=[out4,out3])
display(box2)

In [ ]:
country_vaccine_df = vaccine_df[vaccine_df['State']=='India']
country_vaccine_df.iloc[:,0] = pd.to_datetime(country_vaccine_df.iloc[:,0], format="%d/%m/%Y")
# country_vaccine_df['Updated On'] = pd.to_datetime(country_vaccine_df['Updated On'])

# country_vaccine_df.head()

india_vaccine_df = country_vaccine_df[country_vaccine_df['State'] == 'India']

dose_df = india_vaccine_df[['Updated On', 'First Dose Administered', 'Second Dose Administered']]

dose_df = pd.melt(dose_df, id_vars = 'Updated On', var_name = 'Dose')

## Vaccination spread trends

In [ ]:
hover = alt.selection_single(on='mouseover')

alt.Chart(dose_df).mark_area().encode(
alt.X('Updated On:T'),
alt.Y('sum(value):Q', axis=alt.Axis(title='vaccine dose')),
color=alt.Color('Dose:N',scale = alt.Scale(scheme='dark2')),opacity=alt.condition(hover, alt.value(1.0), alt.value(0.5))
).properties(
    width=600,
    height=400,
    selection=hover
).configure(background='black').interactive()

In [ ]:
gender_vaccine_df = india_vaccine_df[['Male(Individuals Vaccinated)','Female(Individuals Vaccinated)', 'Transgender(Individuals Vaccinated)','Total Covaxin Administered','Total CoviShield Administered']]

gender_vaccine_df.columns = ['Male','Female','Transgender','Covaxin','CoviShield']

vaccine_total_df = gender_vaccine_df.sum(axis=0)

vaccine_type_df= vaccine_total_df[['Covaxin','CoviShield']]

vaccine_type_df =pd.DataFrame(vaccine_type_df)
vaccine_type_df.reset_index(inplace=True)

vaccine_type_df.columns = ['Vaccine','Doses']

In [ ]:
alt.Chart(vaccine_type_df).mark_bar().encode(x='Doses:Q',y='Vaccine:N',color=alt.Color('Vaccine',scale=alt.Scale(scheme='set2'))).properties(height=100,width=600).configure(background='black').interactive()